In [64]:
import warnings

warnings.filterwarnings("ignore")

import os
import sys
import time

import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import words

## Part a: Extracting Language Features

In [65]:
# the interview data (sheet 1)
interview_data = pd.read_excel(
    "../data/DAIC_demographic_data.xlsx",
    sheet_name="Interview_Data",
    skiprows=lambda x: x == 1,
)
# drop the rows where data in column Partic# is NaN
interview_data = interview_data.dropna(subset=["Partic#"])
print(interview_data.tail())

# the phq score data (sheet 2)
phq_score = pd.read_excel(
    "../data/DAIC_demographic_data.xlsx", sheet_name="Metadata_mapping"
)

print(phq_score.head())

     Partic# Condition  gender  race
428    837.0        AI     2.0     3
429    838.0        AI     1.0     1
430    839.0        AI     1.0     1
431    840.0        AI     2.0     3
432    841.0        AI     1.0     4
   Participant_ID  PHQ_Score
0             300          2
1             301          3
2             302          4
3             303          0
4             304          6


The following step was a workaround to prevent the end of file error due to unclosed inverted commas. What the code is doing - opening and reading each file, reading each line and checking for lines that start with " but does not end with " (i.e., checking for unclosed quotes).

In [66]:
# preprocessing the csv files to remove the unclosed inverted commas
def clean_csv(file_path, output_path):
    with open(file_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            if line.startswith('"') and not line.endswith('"\n'):
                line = line[1:]
            outfile.write(line)


def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".csv"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            clean_csv(input_path, output_path)


input_dir = "../data/E-DAIC_Transcripts"
output_dir = "../data/E-DAIC_Transcripts_cleaned"

process_directory(input_dir, output_dir)

In [67]:
cleaned_transcripts_path = "../data/E-DAIC_Transcripts_cleaned"

transcripts = {}

# loop through each file in the folder, load it, and store the content
for filename in os.listdir(cleaned_transcripts_path):
    if filename.endswith("_Transcript.csv"):
        participant_id = filename.split("_")[0]  # extract the participant id
        file_path = os.path.join(cleaned_transcripts_path, filename)

        df = pd.read_csv(file_path)
        transcripts[participant_id] = df

# transcripts
transcripts["386"]

,Text
0,might have pulled something that
1,I'm going to bring the great thanks so much
2,and please
3,are you okay with this yes
4,oh I'm fine I'm a little tired but I found ou...
...,...
76,yeah well after college people usually many p...
77,thank you goodbye
78,oh that was that was fast
79,but I didn't never said there wasn't any like...


In [68]:
# combine all the text data for each participant into a single string
combined_transcripts = {}

for participant_id, transcript in transcripts.items():
    combined_transcripts[participant_id] = " ".join(transcript["Text"].astype(str))

# removing extra spaces caused by newlines
for participant_id, transcript in combined_transcripts.items():
    combined_transcripts[participant_id] = " ".join(transcript.split())

# combined_transcripts

for participant_id, transcript in list(combined_transcripts.items())[:3]:
    print(participant_id, transcript)

423 okay and please yes feeling well where are you from originally Los Angeles the Greater Los Angeles area people diversity and various entertainment and activities fickle weather traffic and litter do you travel I have travel domestically not internationally seeing other places and how people live in the culture I want to hear about one of your trips the last trip that I went to a San Diego butt out from childhood or as an adult family vacations to Baja in Rosarito Beach can you tell me yeah the reason why that memory comes to mind because this weekend is Easter and we usually will go down to East Easter spring break to Rosarito Baja am I have a Bachelors in communication studies because of the open field just doesn't deal with them communication interactively but Performing Arts telecommunications business entrepreneurship so no I'm not at Maxey unemployed what's your dream job I think 200 my business that's my dream my dream job to have my own company I would like to open something

In [69]:
# combine the demographic data with the extracted text data

from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# create the feature extractor objects here; using the base settings for now

corpus = list(combined_transcripts.values())
tfidf_vectorizer = TfidfVectorizer(stop_words="english", token_pattern=r"\b[A-Za-z]+\b")
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

sentiment_analyzer = SentimentIntensityAnalyzer()

combined_data = []

# convert the participant id in demographic data to int for consistency
interview_data["Partic#"] = interview_data["Partic#"].astype(int)


for _, row in interview_data.iterrows():
    participant_id = str(row["Partic#"])  # convert id to match the transcript ids

    # find langauge features for this participant
    if participant_id in combined_transcripts:
        transcript = combined_transcripts[participant_id]

        # sentiment features
        sentiment_pos_scores = sentiment_analyzer.polarity_scores(transcript)["pos"]

        # sentiment features
        sentiment_neu_scores = sentiment_analyzer.polarity_scores(transcript)["neu"]

        # sentiment features
        sentiment_neg_scores = sentiment_analyzer.polarity_scores(transcript)["neg"]

        # combine all the features
        data = {
            "participant_id": participant_id,
            "condition": row["Condition"],
            "race": row["race"],
            "gender": row["gender"],
            # "tfidf_features": tfidf_features,
            # "count_features": count_features,
            "pos_sentiment": sentiment_pos_scores,
            "neu_sentiment": sentiment_neu_scores,
            "neg_sentiment": sentiment_neg_scores,
        }

        combined_data.append(data)

# convert the combined data into a dataframe
combined_data_df = pd.DataFrame(combined_data)

# print(combined_data_df.head())

# create dataframes for tf-idf and count features
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
)

# concatenate the original DataFrame with the TF-IDF and Count DataFrames
final_df = pd.concat([combined_data_df.reset_index(drop=True), tfidf_df], axis=1)

final_df.head()

,participant_id,condition,race,gender,pos_sentiment,neu_sentiment,neg_sentiment,aa,aau,abandoned,...,zip,ziplining,zipping,zodiac,zoloft,zombie,zone,zuma,zurich,zz
0,386,WoZ,3,2.0,0.184,0.770,0.046,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,387,WoZ,1,1.0,0.285,0.665,0.050,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,388,WoZ,4,1.0,0.161,0.769,0.070,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,389,WoZ,1,1.0,0.116,0.827,0.057,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,390,WoZ,3,1.0,0.193,0.740,0.067,0.0,0.0,0.055628,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- By removing common stop words and number, we went from $8702$ features to $8299$ features. We need to investigate how this affects the accuracy.
- Each term in the `TF-IDF` vector is considered a feature. The values represent the `TF-IDF` score for that term. A higher score could mean that the term is important to the transcript but not common in the entire list of transcripts. This is helpful in identifying the transcript's topic/sentiment.
- Initially, I decided on using the compound score, but that resulting list of scores are heavily skewed. Figuring something might be wrong, I reverted back to using individual metrics (positive, negative and neutral)

I noticed that using `CountVectorizer` just adds $8299$ more columns to the dataframe, making the already sparse dataset more sparse. Therefore, I decided to not use it.

In [70]:
# looking at the top 10 words with the highest tf-idf scores

feature_names = tfidf_vectorizer.get_feature_names_out()

# Sum the TF-IDF scores for each feature across all documents
tfidf_scores = np.sum(tfidf_matrix.toarray(), axis=0)

# Create a DataFrame with feature names and their corresponding scores
tfidf_scores_df = pd.DataFrame({"feature": feature_names, "score": tfidf_scores})

# Sort the DataFrame by scores in descending order
tfidf_scores_df = tfidf_scores_df.sort_values(by="score", ascending=False)

# Get the top 10 features
top_10_features = tfidf_scores_df.head(10)

# Display the top 10 features
print(top_10_features)

     feature      score
6326       s  35.261808
7293       t  32.736003
4428       m  29.000342
4287    like  24.292047
4058    just  23.032052
4129    know  22.186805
2193     don  17.789652
5934  really  16.630834
3160    good  12.510597
7451   think  10.710480


In [71]:
corpus = list(combined_transcripts.values())

# create a TfidfVectorizer without removing stop words
vectorizer_no_stop_words = TfidfVectorizer(token_pattern=r"\b[A-Za-z]+\b")
vectorizer_no_stop_words.fit(corpus)
vocab_no_stop_words = set(vectorizer_no_stop_words.get_feature_names_out())

# create a TfidfVectorizer with stop_words='english'
vectorizer_with_stop_words = TfidfVectorizer(
    stop_words="english", token_pattern=r"\b[A-Za-z]+\b"
)
vectorizer_with_stop_words.fit(corpus)
vocab_with_stop_words = set(vectorizer_with_stop_words.get_feature_names_out())

# find the difference between the two vocabularies
removed_stop_words = vocab_no_stop_words - vocab_with_stop_words

print("Removed stop words number:", len(removed_stop_words))

Removed stop words number: 272


In [73]:
# creating 5 folds with shuffled data

from sklearn.model_selection import KFold

# initialize the KFold object with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

folds = []

# split the DataFrame into 5 folds
for train_index, test_index in kf.split(final_df):
    train_fold = final_df.iloc[train_index]
    test_fold = final_df.iloc[test_index]
    folds.append((train_fold, test_fold))

# display the first fold
# train_fold, test_fold = folds[0]
# print("Train Fold:")
# print(train_fold.head())
# print("\nTest Fold:")
# print(test_fold.head())

for i, (train_fold, test_fold) in enumerate(folds):
    print(f"Fold {i+1}:")
    print(f"Train Fold Size: {len(train_fold)}")
    print(f"Test Fold Size: {len(test_fold)}\n")

Fold 1:
Train Fold Size: 152
Test Fold Size: 38

Fold 2:
Train Fold Size: 152
Test Fold Size: 38

Fold 3:
Train Fold Size: 152
Test Fold Size: 38

Fold 4:
Train Fold Size: 152
Test Fold Size: 38

Fold 5:
Train Fold Size: 152
Test Fold Size: 38



### Optional: Doing more preprocessing to remove non-English words

_I used the NLTK library to exclude non-English words from the corpus. This reduces the number of features significantly, as you'll see below. I'm still not sure about removing the stop words though; you can try and see if disabling it helps with the performance._

_You can use this dataset to do the rest of the tasks. I only included this because I figured it might help with overfitting and consequentially, improve the model accuracies._

_For the folds, I have used the `final_df` dataset, which is the one that has $8306$ features. If you want, you can use the dataframe with the non-English words removed._

In [54]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nltk.download("words")

eng_words = set(words.words())

# preprocessing function


def preprocess_text(text):
    text = re.sub(r"\d+", "", text)  # Remove numbers
    tokens = re.findall(r"\b\w+\b", text)  # Tokenize
    valid_words = [
        token for token in tokens if token.lower() in eng_words
    ]  # Filter non-English words
    return " ".join(valid_words)


# create the feature extractor objects here; using the base settings for now

# corpus = list(combined_transcripts.values())
processed_corpus = [preprocess_text(text) for text in corpus]
tfidf_vectorizer_ = TfidfVectorizer(stop_words="english")
tfidf_matrix_ = tfidf_vectorizer_.fit_transform(processed_corpus)

sentiment_analyzer_ = SentimentIntensityAnalyzer()

combined_data_ = []

# convert the participant id in demographic data to int for consistency
interview_data["Partic#"] = interview_data["Partic#"].astype(int)


for _, row in interview_data.iterrows():
    participant_id = str(row["Partic#"])  # convert id to match the transcript ids

    # find langauge features for this participant
    if participant_id in combined_transcripts:
        transcript = combined_transcripts[participant_id]

        # sentiment features
        sentiment_pos_scores = sentiment_analyzer.polarity_scores(transcript)["pos"]

        # sentiment features
        sentiment_neu_scores = sentiment_analyzer.polarity_scores(transcript)["neu"]

        # sentiment features
        sentiment_neg_scores = sentiment_analyzer.polarity_scores(transcript)["neg"]

        # combine all the features
        data = {
            "participant_id": participant_id,
            "condition": row["Condition"],
            "race": row["race"],
            "gender": row["gender"],
            # "tfidf_features": tfidf_features,
            # "count_features": count_features,
            "pos_sentiment": sentiment_pos_scores,
            "neu_sentiment": sentiment_neu_scores,
            "neg_sentiment": sentiment_neg_scores,
        }

        combined_data_.append(data)

# convert the combined data into a dataframe
combined_data_df_ = pd.DataFrame(combined_data_)

# print(combined_data_df.head())

# create dataframes for tf-idf and count features
tfidf_df_ = pd.DataFrame(
    tfidf_matrix_.toarray(), columns=tfidf_vectorizer_.get_feature_names_out()
)

# concatenate the original DataFrame with the TF-IDF and Count DataFrames
final_df_ = pd.concat([combined_data_df_.reset_index(drop=True), tfidf_df_], axis=1)

final_df_.head()

[nltk_data] Downloading package words to
[nltk_data]     /Users/arindrajitpaul/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


,participant_id,condition,race,gender,pos_sentiment,neu_sentiment,neg_sentiment,aa,abandoned,ability,...,youth,yule,zany,zero,zest,zip,zipping,zodiac,zombie,zone
0,386,WoZ,3,2.0,0.184,0.770,0.046,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,387,WoZ,1,1.0,0.285,0.665,0.050,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,388,WoZ,4,1.0,0.161,0.769,0.070,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,389,WoZ,1,1.0,0.116,0.827,0.057,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,390,WoZ,3,1.0,0.193,0.740,0.067,0.0,0.068101,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


By excluding non-english words, along with removing stop words, I was able to reduce the number of features significantly - from $8299$ to $5327$. This may produce better results.

In [57]:
feature_names_ = tfidf_vectorizer_.get_feature_names_out()

# Sum the TF-IDF scores for each feature across all documents
tfidf_scores_ = np.sum(tfidf_matrix_.toarray(), axis=0)

# Create a DataFrame with feature names and their corresponding scores
tfidf_scores_df_ = pd.DataFrame({"feature": feature_names_, "score": tfidf_scores_})

# Sort the DataFrame by scores in descending order
tfidf_scores_df_ = tfidf_scores_df_.sort_values(by="score", ascending=False)

# Get the top 10 features
top_10_features_ = tfidf_scores_df_.head(10)

# Display the top 10 features
print(top_10_features_)

     feature      score
2752    like  29.028222
2612    just  27.465559
2650    know  26.661350
1405     don  21.508594
3770  really  19.885133
2048    good  14.995024
4768   think  12.867584
3400  people  12.553898
4795    time  12.395769
2044   going  11.075604
